<a href="https://colab.research.google.com/github/derek-shing/DS-Unit-4-Sprint-2-NLP/blob/master/DS42SC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 2 Sprint Challenge*

# Natural Language Processing

**Part 1 - Working with Text Data**
Use Python string methods remove irregular whitespace from the following string:

In [2]:
whitespace_string = "\n\n  This is a    string   that has  \n a lot of  extra \n   whitespace.   "

print(whitespace_string)



  This is a    string   that has  
 a lot of  extra 
   whitespace.   


In [7]:
 ##### Your Code Here #####
  
s = " ".join(whitespace_string.split())
s

'This is a string that has a lot of extra whitespace.'

### Use Regular Expressions to take the dates in the following .txt file and put them into a dataframe with columns for:

[RegEx dates.txt](https://raw.githubusercontent.com/ryanleeallred/datasets/master/dates.txt)

- Day
- Month
- Year


In [0]:
##### Your Code Here #####
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/ryanleeallred/datasets/master/dates.txt", header=None)


In [23]:
df.head()

,0,1
0,March 8,2015
1,March 15,2015
2,March 22,2015
3,March 29,2015
4,April 5,2015


In [0]:
import re
data=[]
for i in df[0]:
  month =re.findall("[A-Za-z]+",i)[0]
  day = re.findall("\d+",i)[0]
  d={"Month":month,"Day":day}
  data.append(d)

In [0]:
df2=pd.DataFrame(data)
df2["Year"]=df[1]

In [54]:
df2

,Day,Month,Year
0,8,March,2015
1,15,March,2015
2,22,March,2015
3,29,March,2015
4,5,April,2015
5,12,April,2015
6,19,April,2015
7,26,April,2015
8,3,May,2015
9,10,May,2015


# Part 2 - Bag of Words 

### Use the twitter sentiment analysis dataset found at this link for the remainder of the Sprint Challenge:

[Twitter Sentiment Analysis Dataset](https://raw.githubusercontent.com/ryanleeallred/datasets/master/twitter_sentiment_binary.csv)

 ### Clean and tokenize the documents ensuring the following properties of the text:

1) Text should be lowercase.

2) Stopwords should be removed.

3) Punctuation should be removed.

4) Tweets should be tokenized at the word level. 

(The above don't necessarily need to be completed in that specific order.)

### Output some cleaned tweets so that we can see that you made all of the above changes.


In [0]:
##### Your Code Here #####
df_twit = pd.read_csv("https://raw.githubusercontent.com/ryanleeallred/datasets/master/twitter_sentiment_binary.csv")

In [7]:
df_twit.head()

,Sentiment,SentimentText
0,0,is so sad for my APL frie...
1,0,I missed the New Moon trail...
2,1,omg its already 7:30 :O
3,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,0,i think mi bf is cheating on me!!! ...


In [6]:
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [0]:
df_twit.head()
stop_words = stopwords.words('english')
stop_words = set(stop_words)
table = str.maketrans('','', string.punctuation)

def pre_process(text):
  tokens = word_tokenize(text)
  #print("Tokens:", tokens)
  lowercase_tokens = [w.lower() for w in tokens]
  #print("Lowercase:", lowercase_tokens)
  # Strip punctuation from within words
  no_punctuation = [x.translate(table) for x in lowercase_tokens]
  #print("No Punctuation:", no_punctuation)
  # Remove stopwords
  words = [w for w in no_punctuation if not w in stop_words]
  #print("Cleaned Words:", words)
  #print("--------------------------------")
  return words

In [0]:
df_twit['clean']=df_twit['SentimentText'].apply(pre_process)

In [72]:
df_twit.head()

,Sentiment,SentimentText,clean
0,0,is so sad for my APL frie...,"[sad, apl, friend, , , , , ]"
1,0,I missed the New Moon trail...,"[missed, new, moon, trailer, ]"
2,1,omg its already 7:30 :O,"[omg, already, 730, ]"
3,0,.. Omgaga. Im sooo im gunna CRy. I'...,"[, omgaga, , im, sooo, im, gunna, cry, , denti..."
4,0,i think mi bf is cheating on me!!! ...,"[think, mi, bf, cheating, , , , tt]"


In [13]:
df_twit['clean'].sample(10)

76722                     [, butterjelly, argh, , sorry, ]
73939    [, annamori, reschedule, , free, tmrw, unfort,...
75412    [, bubbleloveee, job, analysis, thing, de, la,...
70496    [, andrewortolano, yikes, , sympathize, , send...
41942     [, andrewkjs, thanx, man, , catch, next, week, ]
76379      [, caseycakessxoxo, , lol, yeeppp, ohmygod, , ]
79108                                  [, cam001, courage]
83570                               [, chiefjudy, notiert]
13334    [, first, jay, havin, dinner, , 4, peaks, surp...
22638    [, 09casper, member, saying, , well, stop, sne...
Name: clean, dtype: object

### How should TF-IDF scores be interpreted? How are they calculated?

#### Your Answer Here #####

TF-IDF scores is about how important of a word in a document. The occurance of the term in the target document will increase the TF-IDF scores while the occurance of the term in other document will decrease the scores.
(in markdown)

# Part 3 - Document Classification

1) Use Train_Test_Split to create train and test datasets.

2) Vectorize the tokenized documents using your choice of vectorization method. 

 - Stretch goal: Use both of the methods that we talked about in class.

3) Create a vocabulary using the X_train dataset and transform both your X_train and X_test data using that vocabulary.

4) Use your choice of binary classification algorithm to train and evaluate your model's accuracy. Report both train and test accuracies.

 - Stretch goal: Use an error metric other than accuracy and implement/evaluate multiple classifiers.
 - Stretch goal: Track your results in a DataFrmae and produce a visualization of the results



In [0]:
##### Your Code Here #####

from sklearn.model_selection import train_test_split

X = df_twit['SentimentText']
y = df_twit['Sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features=None, ngram_range=(1,1), stop_words='english')

vectorizer.fit(X_train)

print(vectorizer.vocabulary_)

print(len(vectorizer.vocabulary_))

{'bellazombie': 27417, 'sleep': 79710, 'suggest': 81960, 'watching': 87682, 'quot': 74995, 'girlfriend': 56904, 'experience': 54053, 'ameeraaa__': 13676, 'ii': 60445, 'school': 77956, 'haha': 58148, 'just': 62799, 'reminding': 76029, 'case': 38423, 'forgot': 55580, 'la': 64038, 'love': 65610, 'country': 47777, 'music': 68906, 'abeeliever': 5698, 'gmornin': 57075, 'little': 65116, 'madeleine': 66090, 'cake': 36494, 'gotta': 57386, 'chameleon': 40250, 'dont': 51877, 'know': 63705, 'coffee': 45687, 'changes': 40368, 'colour': 46114, 'lt': 65738, 'comingrightup': 46264, 'gt': 57828, 'cheapspeakers': 41063, 'everybody': 53813, 'rancho': 75245, 'dancing': 50069, 'infectious': 60874, 'pop': 73582, 'twitpic': 85576, 'working': 88908, 'photo': 72811, 'alas': 9867, 'tis': 84193, 'brian_mercurio': 33425, 'wanna': 87566, 'hear': 58753, 'brian': 33404, 'bet': 28136, 'cute': 49770, 'like': 64916, 'voice': 87244, 'anneskie': 17284, 'friend': 55889, 'told': 84385, 'youtube': 89926, 'got': 57367, 'vide

In [0]:
train_word_counts = vectorizer.transform(X_train)

X_train_vectorized = pd.DataFrame(train_word_counts.toarray(), columns=vectorizer.get_feature_names())

print(X_train_vectorized.shape)
X_train_vectorized.head()

In [2]:
test_word_counts = vectorizer.transform(X_test)

X_test_vectorized = pd.DataFrame(test_word_counts.toarray(), columns=vectorizer.get_feature_names())

#print(X_test_vectorized.shape)
X_test_vectorized.head()

NameError: ignored

In [0]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression(random_state=42).fit(X_train_vectorized, y_train)

train_predictions = LR.predict(X_train_vectorized)
test_predictions = LR.predict(X_test_vectorized)

In [0]:
from sklearn.metrics import accuracy_score

print(f'Train Accuracy: {accuracy_score(y_train, train_predictions)}')
print(f'Test Accuracy: {accuracy_score(y_test, test_predictions)}')

# Part 4 -  Word2Vec

1) Fit a Word2Vec model on your cleaned/tokenized twitter dataset. 

2) Display the 10 words that are most similar to the word "twitter"

In [0]:
##### Your Code Here #####
from gensim.models.word2vec import Word2Vec

In [0]:
model = Word2Vec(df_twit['clean'], min_count=1, size=5)

In [14]:
model.wv.most_similar("twitter")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('bugzyblue', 0.9960994720458984),
 ('spproductions', 0.9953622221946716),
 ('basset', 0.9953151941299438),
 ('ypulse', 0.9952502250671387),
 ('ariannaskye', 0.9947385191917419),
 ('fonder', 0.9946327209472656),
 ('donâ´t', 0.9937576055526733),
 ('ayumirollan', 0.9936709403991699),
 ('tweetin', 0.9936505556106567),
 ('busyinthejing', 0.9935659170150757)]